# 맞춤법 검사

### hanspell 모듈의 특징
* 문장 마지막 띄어쓰기, 줄바꿈 문자 제거
* 처음 시작 띄어쓰기는 유지
* 띄어쓰기나, \n만 있을경우 제거

In [49]:
from hanspell import spell_checker

# # TEST
# try:
#     result = spell_checker.check('\t')
# except Exception as e:
#     print(e)

# print(result.as_dict())

### 1. 맞춤법 검사 파일 경로와 저장 파일 경로 입력

In [50]:
# PATH
# INPUT
SPELL_UNCHECKED_FILE_PATH = "test.txt"#"rsc/training_data/final_text.txt"

# OUTPUT
SPELL_CHECKED_FILE_PATH = "result_test.txt"#"rsc/training_data/final_text_Kospell_check.txt"

### 2. Data 전처리
* 500이내의 문자열만 맞춤법 검사 가능
* 500이상 띄어쓰기가 없는 Data 거르기 -> fuckData 리스트에 인덱스 저장

In [57]:
## 500길이 이하의 문자열로 자르고 500개 이상의 띄어쓰기가 있으면 뺀다.

unchecked_file = open(SPELL_UNCHECKED_FILE_PATH, 'r')
# checked_file = open(SPELL_CHECKED_FILE_PATH, 'w')

lines = unchecked_file.readlines()

temp_str = ""
ready_list = []

# 띄어쓰기가 심각하게 되어있지않는 데이터(500자 이상)
max_iter=1000
debug = 0
flag = True
fuckData = []

# 맞춤범 검사를 하기 위해 길이가 500이하 문자열로 자르기
for i, line in enumerate(lines):
    if i % 10 == 0: print("Processing line num : ", i)
    while(len(line) > 500):
        debug += 1
        if debug > max_iter:
            print("Funking Data!! please check index : {}".format(i))
            print("Didn't insert that data (index : {})".format(i))
            fuckData.append(i)
            break
            
        end = line[:500].rfind(" ")
        temp_str = line[:end]
        ready_list.append(temp_str)
        line = line[end:]
        
    ready_list.append(line)
    debug = 0

print("Finish preprocess for spelling check (Cut string under 500 length)")
print("Total preprocessed list length : {}".format(len(ready_list)))

unchecked_file.close()

Processing line num :  0
Processing line num :  10
Finish preprocess for spelling check (Cut string under 500 length)
Total preprocessed list length : 11


#### 걸러진 데이터 확인, 원본 데이터와 비교

In [ ]:
## 걸러진 fuckData 확인
# fuckData = ((2, 6608), (3, 10540), (3, 16832), (5, 1650), (5, 9128), (5, 12052), (5, 14792), (5, 1650)) // only_paper_v2 결과

for j, (i, n) in enumerate(fuckData):
    print("############ Fuck Data {} ############\n".format(j+1))
    print(paper['PAPER_FULL_TXT'].tolist()[9000*(i-1) + (n//2)])
    print('\n\n\n')

In [ ]:
## 원본 데이터 확인

raw_paper = pd.read_csv("../Dataset/Datacon/paper.csv")
for j, (i, n) in enumerate(fuckData):
    print("############ Fuck Data {} ############\n".format(j+1))
    print(raw_paper['PAPER_FULL_TXT'].tolist()[9000*(i-1) + (n//2)])
    print('\n\n\n')

In [ ]:
### 3. 맞춤법 검사진행
* error 검사 및 * hanspell 모듈이 검사할 수 없는 단어 

In [58]:
# 맞춤법 검사 진행
# 맞춤법 검사 false, 오류가 발생할 경우 검사되지 않은 문자열을 저장(error Message 출력)

spellchecked_li = [] # 검사된 문자열을 담는 리스트
error = False
error_index = []

for i, line in enumerate(ready_list):
    try:
        result = spell_checker.check(line)
    except Exception as e:
        error = True
        error_index.append(i)
        print(e)
        print("Just insert no checked string")
        print("### Error Check This ###\n",line)
        spellchecked_li.append(line)
        continue
    print("processing index num : {}, processing time : {}".format(i, round(result.as_dict()['time'],3)), end='\r',flush=True)
    # drop error 발생시 stop
    if result.as_dict()['result'] == False:
        error = True
        error_index.append(i)
        print("DROP ERROR : error index is {}".format(i))
        print("Just insert no checked string")
        print("### Error Check This ###\n",line)
        spellchecked_li.append(line)
        continue
    spellchecked_li.append(result.as_dict()["checked"])
    
if error: print("\nPlese check error\nerror index numbers : {}".format("\t".join(map(str, error_index))))
else:
    print("\nThere is no error\nContinue file writing")
    
for i, data in enumerate(spellchecked_li):
    if data == '':
        spellchecked_li[i] = '\n\n'

full_text = "".join(spellchecked_li)

checked_file = open(SPELL_CHECKED_FILE_PATH, 'w')
checked_file.write(full_text)
checked_file.close()
print("Complete task : create spell checked text file : {}".format(SPELL_CHECKED_FILE_PATH))

processing index num : 10, processing time : 0.018
There is no error
Continue file writing
Complete task : create spell checked text file : result_test.txt


In [11]:
nbsp = 0
ensp = 0
emsp = 0

for i in range(len(raw_paper)):
    text = raw_paper['PAPER_FULL_TXT'].tolist()[i]
    if "&nbsp;" in text: nbsp += 1
    if "&ensp;" in text: ensp += 1
    if "&emsp;" in text: emsp += 1
        
print(nbsp)
print(ensp)
print(emsp)

23607
2
3


In [60]:
! python spellingCheck.py test.txt --checked_file='result.txt'

Processing line num :  0
Finish preprocess for spelling check (Cut string under 500 length)
Total preprocessed list length : 11
processing index num : 10, processing time : 0.012
There is no error
Continue file writing
Complete task : create spell checked text file : result.txt
